# Alpha Separation
We have a picture of a piece of metal. In the image are two types of alpha crystal - Primary alpha are round large blobs and secondary alpha is the smaller needle like crystals. The objective to to create an image mask to separate the two types of crystal

In [ ]:
from skimage import io, filters, color, measure
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display
import numpy as np
import scipy

plt.rcParams['figure.figsize'] = [12, 8]

## Import image
It's stored as RGBA so convert to greyscale

In [ ]:
image = io.imread("BSE 200um FW.png")
gray_image = color.rgb2gray(color.rgba2rgb(image))
io.imshow(gray_image)
plt.show()

## Crop image
Remove white borders

In [ ]:
x_min = 0
x_max = 1093 # 200 #1093
y_min = 21
y_max = 843 # 221 # 843
gray_image = gray_image[y_min:y_max, x_min:x_max]
io.imshow(gray_image)
plt.show()

## What is interesting to the human eye about this image?
This will inform our approach for the analysis.

One notable aspect is that the primary alpha tends to be more oval while the rods have a much higher aspect ratio. There are some long blobs but these are on a much larger scale. On average the blobs are bigger than the rods.

There is some white background that is neither blobs or rods. All rods are close to some white background but the center of blobs is not close to any white.

## Attempt 1: Characterise distance of pixels from white background
- First threshold the image to get the white particles and non white particles
- Then for all non-white pixels, measure the average distance to the nearest half dozen white pixels

### Filter image

In [ ]:
def threshold_image_display(image: np.ndarray, threshold: int):
    print(image.shape)
    mask = image > threshold
    masked_image = image.copy()
    masked_image[mask] = 0
    io.imshow(masked_image, cmap="gray")
    plt.show()
    print(f"{image.size} total pixels")
    print(f"{np.sum(mask)} background pixels")

In [ ]:
interact(threshold_image_display, image=fixed(gray_image), threshold=widgets.FloatSlider(min=0, max=1, step=1/256, continuous_update=False))

Sliding the slider to 0.45 pretty much separates the alpha and the background.

### Measure distance from each pixel to the background pixels

In [ ]:
background_pixel_locations = np.column_stack(np.where(gray_image > 0.45))
print(background_pixel_locations.shape)

In [ ]:
#TODO: limit the cdist to a 50x50 pixel radius, since no pixel is > 50 pixels from a background pixel.

num_pixels = 5

new_image = []
for x in range(200):
    for y in range(200):
        a = np.array([x, y]).reshape(-1, 2)
        distances = scipy.spatial.distance.cdist(a, background_pixel_locations, 'euclidean')[0]
        # Get the indices of the n closest background pixels
        shortest_indices = np.argpartition(distances, num_pixels)[:num_pixels]
        distance_to_background = np.mean(distances[shortest_indices])
        new_image.append(distance_to_background)

In [ ]:
distance = np.array(new_image).reshape(200, 200)
io.imshow(distance)

Now threshold again - primary alpha is bright

In [ ]:
primary_alpha = distance.copy()
primary_alpha_indices = np.column_stack(np.where(primary_alpha > 7))
plt.imshow(gray_image, cmap='gray')
plt.scatter(primary_alpha_indices[:, 1], primary_alpha_indices[:, 0], s=50, alpha=0.1)
plt.show()

This is pretty good - may need to dilate a little based on the background